In [2]:
from qick import *

qick_path = "/home/xilinx/jupyter_notebooks/qickdawg/firmware/photon_counting/qick_4x2.bit"
ns_host = "172.16.26.2"


In [ ]:
import Pyro4
from qick import QickSoc, QickConfig

ns_port = 8888

server_name = "myqick"

print("looking for nameserver . . .")
Pyro4.config.REQUIRE_EXPOSE = False
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.SERIALIZERS_ACCEPTED=set(['pickle'])
Pyro4.config.PICKLE_PROTOCOL_VERSION=4
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
print("found nameserver")

# if we have multiple network interfaces, we want to register the daemon using the IP address that faces the nameserver
host = Pyro4.socketutil.getInterfaceAddress(ns._pyroUri.host)
daemon = Pyro4.Daemon(host=host)

# if you want to use a different firmware image or set some initialization options, you would do that here
soc = QickSoc(bitfile=qick_path, force_init_clks=True)
print("initialized QICK")
print(QickConfig(soc.get_cfg()))
# register the QickSoc in the daemon (so the daemon exposes the QickSoc over Pyro4)
# and in the nameserver (so the client can find the QickSoc)
ns.register(server_name, daemon.register(soc))
print("registered QICK")

# register in the daemon all the objects we expose as properties of the QickSoc
# we don't register them in the nameserver, since they are only meant to be accessed through the QickSoc proxy
# https://pyro4.readthedocs.io/en/stable/servercode.html#autoproxying
# https://github.com/irmen/Pyro4/blob/master/examples/autoproxy/server.py
for obj in soc.autoproxy:
    daemon.register(obj)
    print("registered member "+str(obj))
    
print("starting daemon")
daemon.requestLoop() # this will run forever until interrupted

looking for nameserver . . .
found nameserver


initialized QICK

QICK configuration:

	Board: RFSoC4x2

	Global clocks (MHz): tProcessor 350.000, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 0, envelope memory 65536 samples
		DAC tile 0, blk 0, 32-bit DDS, fabric=614.400 MHz, f_dds=9830.400 MHz
	1:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, blk 0, 32-bit DDS, fabric=614.400 MHz, f_dds=9830.400 MHz

	2 readout channels:
	0:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, blk 0, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 4, tProc input 0
	1:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, blk 1, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 5, tProc input 1

	2 DACs:
		DAC tile 0, blk 0 is DAC_B
		DAC tile 2, blk 0 is DAC_A

	2 ADCs:
		ADC tile 0, blk 0 is ADC_D
		ADC tile 0, blk 1 is ADC_C

	4 digital output pins (tProc output 7):
	0:	PMO